In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim # 함수 최적화
import numpy as np

In [39]:
import torch

# 시드 설정
torch.manual_seed(1)

# 입력값과 정답값 정의 (2차원 텐서로 정의)
x_data = torch.FloatTensor([[1], [2], [3], [4], [5]])  # 입력값
t_data = torch.FloatTensor([[3], [5], [7], [9], [11]])  # 정답값

# 데이터 확인
print("x_data:", x_data)
print("t_data:", t_data)

x_data: tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.]])
t_data: tensor([[ 3.],
        [ 5.],
        [ 7.],
        [ 9.],
        [11.]])


In [40]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)  # 단순 선형 회귀이므로 input_dim=1, output_dim=1.

    def forward(self, x):
        return self.linear(x)

In [41]:
model=LinearRegressionModel()
print(model)
optimizer=optim.SGD(model.parameters(), lr=0.01)

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)


In [42]:
epochs=2001 #전체 데이터 셋을 몇번 학습할 것인가

for epoch in range(epochs):
    y=model(x_data)
    cost=F.mse_loss(y,t_data) #MSE 손실함수 계산 (평균 제곱 오차)

    optimizer.zero_grad() # 안하면 기존 기울기에 더해지므로 0으로 초기화
    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print(epoch, cost.item(), list(model.parameters())) # 학습 횟수, 손실값, 기울기와 절편 값 


0 39.16685104370117 [Parameter containing:
tensor([[0.9284]], requires_grad=True), Parameter containing:
tensor([-0.3235], requires_grad=True)]
100 0.08463062345981598 [Parameter containing:
tensor([[2.1882]], requires_grad=True), Parameter containing:
tensor([0.3204], requires_grad=True)]
200 0.04298952594399452 [Parameter containing:
tensor([[2.1342]], requires_grad=True), Parameter containing:
tensor([0.5157], requires_grad=True)]
300 0.021837208420038223 [Parameter containing:
tensor([[2.0956]], requires_grad=True), Parameter containing:
tensor([0.6548], requires_grad=True)]
400 0.01109258271753788 [Parameter containing:
tensor([[2.0681]], requires_grad=True), Parameter containing:
tensor([0.7540], requires_grad=True)]
500 0.00563468411564827 [Parameter containing:
tensor([[2.0486]], requires_grad=True), Parameter containing:
tensor([0.8246], requires_grad=True)]
600 0.0028622171375900507 [Parameter containing:
tensor([[2.0346]], requires_grad=True), Parameter containing:
tensor([0

In [43]:
predict =model(torch.FloatTensor([[7]])) # 2차원형태로 넣어줘야함

In [44]:
import pandas as pd 
df=pd.read_csv('../data/data-01-test-score.csv', header=None) # 헤더가 없는 데이터
x_data=df.values[:,:-1] #0부터 마지막 전까지
x_data
t_data=df.values[:,-1] #마지막 열만
print(x_data.shape, t_data.shape)
t_data=t_data.reshape(-1, 1) #-1은 행을 자동으로 계산하라는 의미
print(x_data.shape, t_data.shape)
print(type(x_data), type(t_data)) # 훈련시키기 위해 텐서로 변환해야함

x_data1=torch.from_numpy(x_data).float()
t_data1=torch.from_numpy(t_data).float()
print(type(x_data1), type(t_data1))

indim=x_data1.size(1) # 0이면 2차원의 값, 1이면 1차원의 값
print(indim)

(25, 3) (25,)
(25, 3) (25, 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
3


In [49]:
class MultiLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(indim, 1)

    def forward(self, x):
        return self.linear(x)

In [50]:
model=MultiLinearRegressionModel()
criterion = nn.MSELoss()
optimizer=optim.SGD(model.parameters(), lr=1e-6)

In [51]:
epochs = 20001

# 학습 루프
for epoch in range(epochs):
    # 예측
    y = model(x_data1)
    
    # 손실 계산
    cost = criterion(y, t_data1)
    
    # 역전파
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{epochs}], Loss: {cost.item():.4f}')
        print(f'Weight: {model.linear.weight.data.numpy()}, Bias: {model.linear.bias.data.numpy()}')

Epoch [0/20001], Loss: 20483.0645
Weight: [[-0.09591647  0.31669506  0.10380485]], Bias: [-0.07040991]
Epoch [100/20001], Loss: 20.2364
Weight: [[0.45392194 0.8682395  0.67179894]], Bias: [-0.06359691]
Epoch [200/20001], Loss: 13.2856
Weight: [[0.46369904 0.8767572  0.6840848 ]], Bias: [-0.06347523]
Epoch [300/20001], Loss: 13.2201
Weight: [[0.46356934 0.8753202  0.68616766]], Bias: [-0.06347632]
Epoch [400/20001], Loss: 13.1575
Weight: [[0.4632589 0.8737076 0.6880556]], Bias: [-0.06347996]
Epoch [500/20001], Loss: 13.0955
Weight: [[0.46294597 0.87209976 0.6899308 ]], Bias: [-0.06348369]
Epoch [600/20001], Loss: 13.0340
Weight: [[0.46263352 0.87049925 0.69179875]], Bias: [-0.06348741]
Epoch [700/20001], Loss: 12.9730
Weight: [[0.4623229  0.8689058  0.69365776]], Bias: [-0.06349114]
Epoch [800/20001], Loss: 12.9126
Weight: [[0.46201295 0.8673196  0.6955086 ]], Bias: [-0.06349486]
Epoch [900/20001], Loss: 12.8528
Weight: [[0.46170393 0.8657412  0.6973522 ]], Bias: [-0.06349859]
Epoch [10

In [52]:
x=torch.FloatTensor([[75,85,90],[90,85,75]])
model(x)

tensor([[172.0592],
        [164.2733]], grad_fn=<AddmmBackward0>)